In [1]:
simbolo_test = "BTG"
simbolo_validazione = "DHT"
n_simboli_addestramento = 10
epochs = 50
batch_size = 32
#learning_rate = 0.001
n_timesteps = 60 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 10 # n. barre nel futuro di cui si desidera prevedere il prezzo
seed = 10 
elenco_features = [
    #"Close",
    "EMA_5", 
    "EMA_20", 
    "EMA_50",
    #"Open",  
    #"High",
    #"Low",
    "Volume",
    "MACDh",
    "PSAR",
    "PSARaf",
    "SUPERT", 
    "TRIX",
    "ATR"
]
elenco_target = [
    #"Close"
    "EMA_5",
    "EMA_20", 
    "EMA_50"    
]

col_features = {col: idx for idx, col in enumerate(elenco_features)}
col_targets = {col: idx for idx, col in enumerate(elenco_target)}
n_features = len(col_features)
n_targets = len(col_targets)

def crea_modello(giorni_previsione=giorni_previsione, n_targets=n_targets, n_features=n_features, neuroni=50, l2_rate=0, dropout_rate=0, optimizer='adam'):
    model = Sequential([
        LSTM(neuroni, input_shape=(n_timesteps, n_features)),
        RepeatVector(giorni_previsione),
        LSTM(neuroni, return_sequences=True, kernel_regularizer=l2(l2_rate)),
        Dropout(dropout_rate),
        #LSTM(100, return_sequences=True, kernel_regularizer=l2(0.01)),
        #Dropout(0.2),
        #LSTM(50, kernel_regularizer=l2(0.01)),
        #Dropout(0.2),
        #Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        #Dropout(0.2),
        TimeDistributed(Dense(n_targets, activation='linear'))
    ])
    model.compile(optimizer=optimizer, loss='mean_absolute_percentage_error', metrics=['mse', 'mae', 'msle', 'huber_loss'])
    return model

from sklearn.preprocessing import PowerTransformer
#from sklearn.preprocessing import RobustScaler
from sklearn.base import clone
scaler = PowerTransformer()
#scaler = RobustScaler()

from skopt.space import Real, Categorical, Integer
param_space = {
    'neuroni': Integer(10, 150),
    'dropout_rate': Real(0.0, 0.5),
    'optimizer': Categorical(['adam', 'rmsprop']),
    'batch_size': Integer(10, 50),
    'epochs': Integer(10, 100)
}

In [2]:
print("Importa librerie", end="", flush=True)
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
print(".", end="", flush=True)
import pandas as pd
print(".", end="", flush=True)
import yfinance as yf
print(".", end="", flush=True)
import numpy as np
print(".", end="", flush=True)
import tensorflow as tf
print(".", end="", flush=True)
import fx_com
print(".", end="", flush=True)
import funzioni as fx
print(".", end="", flush=True)
from tensorflow.keras.layers import Dropout
print(".", end="", flush=True)
from tensorflow.keras.layers import RepeatVector
print(".", end="", flush=True)
from tensorflow.keras.layers import TimeDistributed
print(".", end="", flush=True)
from keras.regularizers import l2
print(".", end="", flush=True)
from tensorflow.keras import Sequential
print(".", end="", flush=True)
from tensorflow.keras.layers import LSTM, Dense
print(".", end="", flush=True)
from tensorflow.keras.optimizers import Adam
print(".", end="", flush=True)
from sklearn.model_selection import train_test_split
print(".", end="", flush=True)
from tensorflow.keras.models import load_model
print(".", end="", flush=True)
import os
print(".", end="\n", flush=True)

np.random.seed(seed)
tf.random.set_seed(seed)
pd.set_option("display.max_columns", None)

def ottimizzazione(X_train, Y_train, n_iter=50, cv=3, n_jobs=-1, seed=42):
    from skopt import BayesSearchCV
    model = KerasRegressor(build_fn=crea_modello, epochs=epochs, batch_size=batch_size, verbose=0)
    opt = BayesSearchCV(
        model,
        param_space,
        n_iter=n_iter,
        cv=cv,
        n_jobs=n_jobs,
        random_state=seed
    )
    opt.fit(X_train, Y_train)
    print("Migliori iperparametri:", opt.best_params_)

print("Download lista ticker")
lista_ticker = pd.read_parquet("Tickers_De_Giro.parquet")
lista_ticker = lista_ticker.sample(frac=1).reset_index(drop=True)
lista_ticker = lista_ticker.loc[(lista_ticker["Ticker"] != simbolo_test) & (lista_ticker["Ticker"] != simbolo_validazione) & (lista_ticker["Categoria"] != "D"), :]

if os.path.exists('LSTM.keras'):
    print("Caricamento modello esistente")
    model = load_model("LSTM.keras")
else:
    print("Creazione modello")
    model = crea_modello(giorni_previsione, n_targets, n_features)

def to_XY(dati_ticker, n_timesteps, giorni_previsione, addestramento=True):
    new_dates = pd.bdate_range(start=dati_ticker.index[-1] + pd.Timedelta(days=1), periods=giorni_previsione)
    df_new = pd.DataFrame(index=new_dates)
    dati_ticker = pd.concat([dati_ticker, df_new])

    features = dati_ticker[elenco_features]
    targets = dati_ticker[elenco_target]

    if addestramento:
        i_tot = len(features) - giorni_previsione*2
    else:
        i_tot = len(features) - giorni_previsione
    X, Y = [], []
    for i in range(n_timesteps - 1, i_tot):
        X.append(features.iloc[i - (n_timesteps - 1):i + 1])
        Y.append(targets.iloc[i + 1:i + 1 + giorni_previsione])
    
    idx = dati_ticker.index[n_timesteps - 1:i_tot]
    
    return idx, np.array(X), np.array(Y)


Importa librerie

ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'

In [ ]:
print("Download dati ticker validazione")
ticker_val = yf.download(simbolo_validazione, start='2010-01-01', end='2023-12-31', progress=False)
ticker_val.index = ticker_val.index.date
print("Calcolo indicatori ticker validazione")
ticker_val = fx.crea_indicatori(ticker_val)
ticker_val.dropna(axis=0, inplace=True)

for i_ticker in range (n_simboli_addestramento + 1):
    nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
    print(f"\033[48;5;42m{i_ticker} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
    print("Download dati ticker")
    ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
    if ticker["Close"].iloc[-1] >= 1:
        ticker.index = ticker.index.date
        print("Calcolo indicatori ticker validazione")
        ticker = fx.crea_indicatori(ticker)
        ticker.dropna(axis=0, inplace=True)

        print("Definizione features e target")
        idx, X, Y = to_XY(ticker, n_timesteps, giorni_previsione, addestramento=True)
        idx_val, X_val, Y_val = to_XY(ticker_val, n_timesteps, giorni_previsione, addestramento=True)

        print("Scaler")
        X = X.reshape((-1, n_timesteps * n_features))
        Y = Y.reshape((-1, giorni_previsione * n_targets))
        X_val = X_val.reshape((-1, n_timesteps * n_features))
        Y_val = Y_val.reshape((-1, giorni_previsione * n_targets))

        #X_train, X_test, Y_train, Y_test, idx_train, idx_test = train_test_split(X, Y, idx, test_size=0.2, random_state=seed)
        X_train = X
        Y_train = Y

        X_scaler = clone(scaler)
        Y_scaler = clone(scaler)
        X_scaler_val = clone(scaler)
        Y_scaler_val = clone(scaler)
        X_scaler.fit(X_train)
        Y_scaler.fit(Y_train)
        X_scaler_val.fit(X_val)
        Y_scaler_val.fit(Y_val)
        X_train = X_scaler.fit_transform(X_train)
        #X_test = X_scaler.fit_transform(X_test)
        Y_train = Y_scaler.fit_transform(Y_train)
        #Y_test = Y_scaler.fit_transform(Y_test)
        X_val = X_scaler_val.fit_transform(X_val)
        Y_val = Y_scaler_val.fit_transform(Y_val)

        X_train = X_train.reshape((-1, n_timesteps, n_features))
        X_val = X_val.reshape((-1, n_timesteps, n_features))
        #X_test = X_test.reshape((-1, n_timesteps, n_features))
        Y_train = Y_train.reshape((-1, giorni_previsione, n_targets))
        Y_val = Y_val.reshape((-1, giorni_previsione, n_targets))
        #Y_test = Y_test.reshape((-1, giorni_previsione, n_targets))

        print(f"Addestramento modello epochs={epochs}, batch_size={batch_size}")
        model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val))
        model.save("LSTM.keras")


In [ ]:

print("Download dati ticker per previsione")
dati_previsione = yf.download(simbolo_test, start='2010-01-01', end='2023-12-31', progress=False)
dati_previsione.index = dati_previsione.index.date
dati_previsione = fx.crea_indicatori(dati_previsione)
dati_previsione.iloc[:100] = dati_previsione.iloc[:100].dropna(axis=0)

indice_prev, X_prev, Y_prev = to_XY(dati_previsione, n_timesteps, giorni_previsione, addestramento=False)

X_prev = X_prev.reshape(-1, n_timesteps * n_features)
Y_prev = Y_prev.reshape(-1, giorni_previsione * n_targets)
X_scaler_prev = clone(scaler)
Y_scaler_prev = clone(scaler)
X_prev = X_scaler_prev.fit_transform(X_prev)
Y_prev = Y_scaler_prev.fit_transform(Y_prev)
X_prev = X_prev.reshape(-1, n_timesteps, n_features)
Y_prev = Y_prev.reshape(-1, giorni_previsione, n_targets)

pred = model.predict(X_prev)
#pred_loss, pred_mae = model.evaluate(X_prev, Y_prev)

X_prev = X_prev.reshape(-1, n_timesteps * n_features)
X_prev = X_scaler_prev.inverse_transform(X_prev)
X_prev = X_prev.reshape(-1, n_timesteps, n_features)

pred = pred.reshape(-1, giorni_previsione * n_targets)
pred = Y_scaler_prev.inverse_transform(pred)
pred = pred.reshape(-1, giorni_previsione, n_targets)



In [ ]:
dati_previsione = dati_previsione.loc[indice_prev]
new_dates = pd.bdate_range(start=dati_previsione.index[-1] + pd.Timedelta(days=1), periods=giorni_previsione)
df_new = pd.DataFrame(index=new_dates.date)
dati_previsione = pd.concat([dati_previsione, df_new])

col_analisi = "EMA_20"
risultato = pd.DataFrame({"Previsione": pred[:, giorni_previsione-1, col_targets[col_analisi]].round(2)}, index=dati_previsione.index[giorni_previsione:])
risultato = pd.concat([dati_previsione[col_analisi], risultato], axis=1)

In [ ]:
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as sp
import yfinance as yf

target = go.Scatter(
    x = risultato.index,
    y = risultato[col_analisi],
    mode = 'lines',
    line = dict(color='rgba(0, 0, 0, .9)'),
    name = col_analisi
)

previsione = go.Scatter(
    x = risultato.index,
    y = risultato['Previsione'],
    mode = 'lines',
    line = dict(color='rgba(0, 0, 250, .9)'),
    name = 'Previsione'
)
'''
err_meno = go.Scatter(
    x = risultato.index,
    y = risultato['Previsione'] - err, 
    mode = 'lines',
    line = dict(
        color='rgba(0, 0, 250, .9)',
        width=1
    ),
    connectgaps = False,
    name = 'err-'
)                
err_piu = go.Scatter(
    x = risultato.index,
    y = risultato['Previsione'] + err, 
    mode = 'lines',
    line = dict(
        color='rgba(0, 0, 250, .9)',
        width=1
    ),
    fill='tonexty',
    fillcolor='rgba(0, 0, 250, .2)', # Puoi cambiare il valore di alpha per regolare la trasparenza
    connectgaps = False,
    name = 'err+'
)
'''
layout = dict(xaxis = dict(autorange=True),
              yaxis = dict(title = 'Previsioni', autorange=True),
              autosize = True,
              margin = go.layout.Margin(
                  l=0,  # Sinistra
                  r=0,  # Destra
                  b=0,  # Basso
                  t=50,  # Alto
                  pad=0  # Padding
              ),
              legend = dict(traceorder = 'normal', bordercolor = 'black')
)
fig = sp.make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.update_layout(layout)
fig.add_trace(target, row=1, col=1)
fig.add_trace(previsione, row=1, col=1)
#fig.add_trace(err_meno, row=1, col=1)
#fig.add_trace(err_piu, row=1, col=1)
pyo.plot(fig, filename="regressione.html", auto_open=True)

